# Notebook 8 - Iroha 1 Blockchain Tools

[From Python Iroha Examples](https://github.com/hyperledger-iroha/iroha-python/tree/1.1.3/examples)


## Get Blocks

[Infinite Block Stream](examples/infinite-blocks-stream.py)

Interrupt execution to stop streaming.

In [ ]:
from integration_helpers import get_blocks


In [ ]:
get_blocks()

The next block arrived: block_response {
  block {
    block_v1 {
      payload {
        transactions {
          payload {
            reduced_payload {
              commands {
                call_engine {
                  caller: "admin@test"
                  callee: "7F527500292905F7FFDABA710453F1EAEFAC9CF7"
                  input: "4518f6b300000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000000000018616E6772795F70726F736B757269616B6F766140746573740000000000000000"
                }
              }
              creator_account_id: "admin@test"
              created_time: 1738270819910
              quorum: 1
            }
          }
          signatures {
            public_key: "313a07e6384776ed95447710d15e59148473ccfc052a681317a72a69f2a49910"
            signature: "92bbd7fa407ff3e61d754b935e9b65ce089db8222e9e70d93f7c4b25e57f21b78c72cd583ee97044555fc1f4341e0defb67eb60cf930b8df506c9e34fcdcea0c"
          }
 

[Get Current Block](https://github.com/hyperledger-iroha/iroha-python/blob/1.1.3/examples/blocks-query.py)

In [ ]:
from iroha import Iroha, IrohaGrpc
from iroha import IrohaCrypto
import os
import sys
import time
import uuid

def send_tx():
    rand_name = uuid.uuid4().hex
    rand_key = IrohaCrypto.private_key()
    domain = ADMIN_ACCOUNT_ID.split('@')[1]
    tx = iroha.transaction([
        iroha.command('CreateAccount',
                      account_name=rand_name,
                      domain_id=domain,
                      public_key=rand_key)
    ])
    IrohaCrypto.sign_transaction(tx, ADMIN_PRIVATE_KEY)
    net.send_tx(tx)
    print('tx is sent')


if __name__ == '__main__':
    query = iroha.blocks_query()
    IrohaCrypto.sign_query(query, ADMIN_PRIVATE_KEY)
    blocks = net.send_blocks_stream_query(
        query, timeout=120)  # timeout in seconds
    send_tx()
    print(next(blocks))

# Get Transactions

In [ ]:
from get_transactions import *

tx_hash, first_time, last_time, ft_p, lt_p = make_initial_transactions()
hash = create_contract()
address = integration_helpers.get_engine_receipts_address(hash)
print('get account transactions results: ')
hash = get_account_transactions(address, 4, 5, '[{"Field": "kCreatedTime", "Direction": "kDescending"},{"Field": "kPosition", "Direction": "kDescending"}]')
integration_helpers.get_engine_receipts_result(hash)
print('get transactions result')
tx_hashes = json.dumps([tx_hash.decode()])
logger.warning(tx_hashes)
hash = get_transactions(address, tx_hashes)
integration_helpers.get_engine_receipts_result(hash)
print('done')

# [Compare and Set Account Detail](https://iroha.readthedocs.io/en/develop/develop/api/commands.html?highlight=compareandsetaccountdetail)


In [ ]:
test_private_key = IrohaCrypto.private_key()
test_public_key = IrohaCrypto.derive_public_key(test_private_key).decode("utf-8")

@integration_helpers.trace
def send_transaction_and_print_status(transaction):
    IrohaCrypto.sign_transaction(transaction, ADMIN_PRIVATE_KEY)
    hex_hash = binascii.hexlify(IrohaCrypto.hash(transaction))
    print('Transaction hash = {}, creator = {}'.format(
        hex_hash, transaction.payload.reduced_payload.creator_account_id))
    response = net.send_tx(transaction)
    print(response)
    for status in net.tx_status_stream(transaction):
        print(status)
    return hex_hash

@integration_helpers.trace
def create_contract():
    bytecode = "608060405234801561001057600080fd5b5073a6abc17819738299b3b2c1ce46d55c74f04e290c6000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff160217905550610772806100746000396000f3fe608060405234801561001057600080fd5b50600436106100365760003560e01c8063d4e804ab1461003b578063da98f54614610059575b600080fd5b610043610089565b6040516100509190610488565b60405180910390f35b610073600480360381019061006e9190610295565b6100ad565b60405161008091906104a3565b60405180910390f35b60008054906101000a900473ffffffffffffffffffffffffffffffffffffffff1681565b6060600086868686866040516024016100ca9594939291906104c5565b6040516020818303038152906040527fda98f546000000000000000000000000000000000000000000000000000000007bffffffffffffffffffffffffffffffffffffffffffffffffffffffff19166020820180517bffffffffffffffffffffffffffffffffffffffffffffffffffffffff8381831617835250505050905060008060008054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16836040516101919190610471565b600060405180830381855af49150503d80600081146101cc576040519150601f19603f3d011682016040523d82523d6000602084013e6101d1565b606091505b509150915081610216576040517f08c379a000000000000000000000000000000000000000000000000000000000815260040161020d9061053b565b60405180910390fd5b80935050505095945050505050565b600061023861023384610580565b61055b565b905082815260208101848484011115610254576102536106cd565b5b61025f848285610626565b509392505050565b600082601f83011261027c5761027b6106c8565b5b813561028c848260208601610225565b91505092915050565b600080600080600060a086880312156102b1576102b06106d7565b5b600086013567ffffffffffffffff8111156102cf576102ce6106d2565b5b6102db88828901610267565b955050602086013567ffffffffffffffff8111156102fc576102fb6106d2565b5b61030888828901610267565b945050604086013567ffffffffffffffff811115610329576103286106d2565b5b61033588828901610267565b935050606086013567ffffffffffffffff811115610356576103556106d2565b5b61036288828901610267565b925050608086013567ffffffffffffffff811115610383576103826106d2565b5b61038f88828901610267565b9150509295509295909350565b6103a5816105f4565b82525050565b60006103b6826105b1565b6103c081856105c7565b93506103d0818560208601610635565b6103d9816106dc565b840191505092915050565b60006103ef826105b1565b6103f981856105d8565b9350610409818560208601610635565b80840191505092915050565b6000610420826105bc565b61042a81856105e3565b935061043a818560208601610635565b610443816106dc565b840191505092915050565b600061045b6027836105e3565b9150610466826106ed565b604082019050919050565b600061047d82846103e4565b915081905092915050565b600060208201905061049d600083018461039c565b92915050565b600060208201905081810360008301526104bd81846103ab565b905092915050565b600060a08201905081810360008301526104df8188610415565b905081810360208301526104f38187610415565b905081810360408301526105078186610415565b9050818103606083015261051b8185610415565b9050818103608083015261052f8184610415565b90509695505050505050565b600060208201905081810360008301526105548161044e565b9050919050565b6000610565610576565b90506105718282610668565b919050565b6000604051905090565b600067ffffffffffffffff82111561059b5761059a610699565b5b6105a4826106dc565b9050602081019050919050565b600081519050919050565b600081519050919050565b600082825260208201905092915050565b600081905092915050565b600082825260208201905092915050565b60006105ff82610606565b9050919050565b600073ffffffffffffffffffffffffffffffffffffffff82169050919050565b82818337600083830152505050565b60005b83811015610653578082015181840152602081019050610638565b83811115610662576000848401525b50505050565b610671826106dc565b810181811067ffffffffffffffff821117156106905761068f610699565b5b80604052505050565b7f4e487b7100000000000000000000000000000000000000000000000000000000600052604160045260246000fd5b600080fd5b600080fd5b600080fd5b600080fd5b6000601f19601f8301169050919050565b7f4572726f722063616c6c696e67207365727669636520636f6e7472616374206660008201527f756e6374696f6e0000000000000000000000000000000000000000000000000060208201525056fea26469706673582212201bf5f5bf321529864a8ed47f18763a099d79ba82774a1d174f40b3361962d3bc64736f6c63430008070033"
    """Bytecode was generated using remix editor  https://remix.ethereum.org/ from file get_transactions.sol. """
    tx = iroha.transaction(
        [iroha.command("CallEngine", caller=ADMIN_ACCOUNT_ID, input=bytecode)]
    )
    return send_transaction_and_print_status(tx)

@integration_helpers.trace
def compare_account_det(address, key, value):
    params = integration_helpers.get_first_four_bytes_of_keccak(b'compareAndSetAccountDetail(string,string,string,string,string)') #See https://iroha.readthedocs.io/en/develop/develop/api/commands.html?highlight=compareandsetaccountdetail
    no_of_param = 5
    for x in range(no_of_param):
        params = params + integration_helpers.left_padded_address_of_param(x, no_of_param)
    params = params + integration_helpers.argument_encoding(ADMIN_ACCOUNT_ID)  # account
    params = params + integration_helpers.argument_encoding(key)  # key
    params = params + integration_helpers.argument_encoding(value)  # value
    params = params + integration_helpers.argument_encoding('')  # key
    # accepted values to check empty: 1, t, T, TRUE, true, True, 0, f, F, FALSE, false, False
    params = params + integration_helpers.argument_encoding('TRUE')  # key  
    tx = iroha.transaction(
        [
            iroha.command(
                "CallEngine", caller=ADMIN_ACCOUNT_ID, callee=address, input=params
            )
        ]
    )
    return send_transaction_and_print_status(tx)
    

hash = create_contract()
address = integration_helpers.get_engine_receipts_address(hash)
print('CompareAndSetAccountDetail result')
hash = compare_account_det(address, 'some_key', 'some_value')
integration_helpers.get_engine_receipts_result(hash)
print('done')
